# 1

In [24]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

training_epochs = 15 # training 반복 횟수
batch_size = 100

root = './data'
mnist_train = dset.MNIST(root = root,train = True, transform = transforms.ToTensor(), download = True)
mnist_test = dset.MNIST(root = root, train = False, transform = transforms.ToTensor(), download = True)

In [25]:
# data loader를 직접 구현해보자
train_loader = DataLoader(mnist_train, batch_size = batch_size, shuffle = True, drop_last = True)
test_loader = DataLoader(mnist_test, batch_size = batch_size, shuffle = True, drop_last = True)

# 2

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = torch.nn.Linear(784, 10, bias = True).to(device)

# weight init
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[ 0.3325,  0.8444, -0.0331,  ...,  0.2162, -1.4177,  1.0080],
        [-2.1705, -0.4159,  0.1210,  ..., -0.2187, -0.3040,  0.2329],
        [ 0.2954,  0.2309, -1.5860,  ...,  1.8515, -1.5457,  1.0488],
        ...,
        [ 0.6200, -1.8023,  0.6401,  ...,  0.8125, -0.0258,  0.3674],
        [-0.1778,  1.3066,  0.5085,  ..., -1.0116, -0.0800, -0.0499],
        [-0.4454,  0.6465, -0.3416,  ...,  0.4601, -0.7384, -2.4744]],
       requires_grad=True)

# 3

In [27]:
# Loss fn - Cross Entropy Loss
criterion = torch.nn.CrossEntropyLoss().to(device)

# optimizer - SGD
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.2) # lr은 자유롭게 설정 가능

# 4

In [28]:
for epoch in range(training_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1,28 * 28)

        outputs = linear(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _,argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss : {:.4f}, Accuracy: {:.2f}%'.format(
                epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item() * 100))

Epoch [1/15], Step [100/600], Loss : 1.7713, Accuracy: 65.00%
Epoch [1/15], Step [200/600], Loss : 1.1455, Accuracy: 73.00%
Epoch [1/15], Step [300/600], Loss : 1.0223, Accuracy: 77.00%
Epoch [1/15], Step [400/600], Loss : 0.7706, Accuracy: 81.00%
Epoch [1/15], Step [500/600], Loss : 0.9017, Accuracy: 72.00%
Epoch [1/15], Step [600/600], Loss : 0.5928, Accuracy: 87.00%
Epoch [2/15], Step [100/600], Loss : 0.9562, Accuracy: 80.00%
Epoch [2/15], Step [200/600], Loss : 1.2893, Accuracy: 78.00%
Epoch [2/15], Step [300/600], Loss : 0.8093, Accuracy: 83.00%
Epoch [2/15], Step [400/600], Loss : 0.6858, Accuracy: 80.00%
Epoch [2/15], Step [500/600], Loss : 0.6598, Accuracy: 82.00%
Epoch [2/15], Step [600/600], Loss : 0.7341, Accuracy: 83.00%
Epoch [3/15], Step [100/600], Loss : 0.2814, Accuracy: 92.00%
Epoch [3/15], Step [200/600], Loss : 0.5720, Accuracy: 90.00%
Epoch [3/15], Step [300/600], Loss : 0.7863, Accuracy: 82.00%
Epoch [3/15], Step [400/600], Loss : 0.7904, Accuracy: 86.00%
Epoch [3

# 5

In [29]:
linear.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        outputs = linear(imgs) # 구현

        _, argmax = torch.max(outputs, 1) # max()를 통해 최종 출력이 가장 높은 class 선택
        total += imgs.size(0)
        correct += (labels == argmax). sum().item()

    print('Test accuracy for {} images: {: .2f}%'.format(total, correct / total * 100))

Test accuracy for 10000 images:  90.54%
